# [RAG - Part 1](https://python.langchain.com/docs/concepts/rag/)

* `Part 1` : **simple Q&A application** over **unstructured data**
* `Part 2` : extends to accommodate **conversation-style interactions** & **multi-step retrieval** processes
  * typical Q&A architecture
  * additional resources for more advanced Q&A techniques

In [5]:
###############
### ~Setup~ ###
###############

import os
from dotenv import load_dotenv

# Load secrets from file
with open('secrets.txt') as f:
    for line in f:
        if '=' in line:
            key, value = line.strip().split('=', 1)
            os.environ[key] = value

# Initialize LangChain
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
llm = ChatOpenAI(model="gpt-4o-mini")

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

RAG has 2 parts:

1. Indexing: pipeline for ingesting data from source & indexing it (usually happens offline)

2. Retrieval & generation: actual **RAG chain**: 
   * takes user query at run time
   * retrieves  relevant data from index 
   * passes that to `model`

> ℹ **Note**: indexing portion of this tutorial will largely follow the `3_retreivers.ipynb`

Most common full sequence from raw data to answer:

#### 1️⃣ Indexing
---
1. **Load**: Done with [Document Loaders](https://python.langchain.com/docs/concepts/document_loaders/).
2. **Split**: [Text splitters](https://python.langchain.com/docs/concepts/text_splitters/) break large Documents into smaller chunks. Useful both for indexing data & passing it into model: **large chunks are harder to search over & won't fit in a model's finite context window**
3. **Store**: Need somewhere to store & index our splits, so that they can be searched over later. Often done usin [VectorStore](https://python.langchain.com/docs/concepts/vectorstores/) & [Embeddings models](https://python.langchain.com/docs/concepts/embedding_models/).

![Image URL](https://python.langchain.com/assets/images/rag_indexing-8160f90a90a33253d0154659cf7d453f.png)

#### 2️⃣ Retreival & Generation
---
4. Retrieve: Given a user input, relevant splits are retrieved from storage using a [Retriever](https://python.langchain.com/docs/concepts/retrievers/)
5. Generate: A [ChatModel](https://python.langchain.com/docs/concepts/chat_models/) / [LLM](https://python.langchain.com/docs/concepts/text_llms/) **produces an answer** using prompt that includes **both the question with retrieved data**
![Image URL](https://python.langchain.com/assets/images/rag_retrieval_generation-1046a4668d6bb08786ef73c56d4f228a.png)

> Once *data indexed*, we will use `LangGraph` as **orchestration framework** to RAG steps.

> #### `InMemoryVectorStore`

In [10]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200)

all_splits = text_splitter.split_documents(docs)

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

# Define application steps


def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke(
        {"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()